In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time
import vision

import building

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
explore = require('./control_primitives/exploreUntil.js')
globalThis.explore = explore
move = require('./control_primitives/goToPosition.js')
kill = require('./control_primitives/killMob.js')
collectPosition = require('./control_primitives/mineBlockPosition.js')
collectType = require('./control_primitives/mineBlockType.js')
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
place = require('./control_primitives/placeItem.js')
shoot = require('./control_primitives/shoot.js')
smelt = require('./control_primitives/smeltItem.js')
chest = require('./control_primitives/useChest.js')

Vec3 = require('vec3').Vec3
mineflayer = require('mineflayer')
pathfinder = require('mineflayer-pathfinder')
pvp = require('mineflayer-pvp').plugin
mcData = require('minecraft-data')(MINECRAFT_VERSION)
globalThis.mcData = mcData
armorManager = require("mineflayer-armor-manager")
autoeat = require('mineflayer-auto-eat').plugin
collectblock = require('mineflayer-collectblock').plugin
hawkeye = require('minecrafthawkeye')
toolPlugin = require('mineflayer-tool').plugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')
  
  return bot, movements

In [2]:
bot, movements = load_bot('poo_bucket176')
globalThis.bot = bot

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [3]:
import memory
import perception
import env_info
# import action

percept = perception.Module()
mem = memory.Module()
# act = action.Module()

c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
from javascript import require, globalThis
findAndParseJsonLikeText = require('json-like-parse')

In [5]:
import action_llms

llms = action_llms.get_llms()
bot_functions = json.loads(open('bot_functions.json').read())

previous_action_outcomes = []
current_goal_progress = ''

In [6]:
# goal = percept.perceive(bot, 'What is the logical, slow next step to achieve in Minecraft?')
goal = 'Collect the nearby wood.'

In [7]:
while True:
    
    while bot.pathfinder.isMoving():
        time.sleep(1)

    # Generate a question to ask the perception module, so we can gather some useful information about the environment
    perception_query = llms['generate_perception_query'].invoke({
        'goal': goal,
        'current_environment': env_info.getPromptInfo(bot),
        'previous_action_outcomes': previous_action_outcomes,
        'current_goal_progress': current_goal_progress
    }).content

    perception_query = findAndParseJsonLikeText(perception_query)[0]['question']

    print(f'{perception_query=}')

    # Ask the perception module the question, and get the answer. Should enlighten us about the environment
    answer = percept.perceive(bot, perception_query)

    failed_action_proposal = ''
    additional_info = ''
    while True:
        try:
            # Generate the next best action to take, given the current environment and the goal
            new_action = llms['generate_next_best_action'].invoke({
                'goal': goal,
                'current_environment': env_info.getPromptInfo(bot),
                'previous_action_outcomes': previous_action_outcomes,
                'current_goal_progress': current_goal_progress,
                'question': perception_query,
                'answer': answer,
                'proposed_action': failed_action_proposal,
                'additional_information': additional_info,
                'bot_functions': bot_functions
            }).content

            print(f'{new_action=}')

            next_best_action = findAndParseJsonLikeText(new_action)[0]['next_best_action']

            print(f'{new_action=}')

            # Ensure this action is proper syntax, makes legitimate sense for our situation, is not a duplicate of the last action or a duplicate of the last 3 actions
            gate = llms['gate_action'].invoke({
                'goal': goal,
                'current_environment': env_info.getPromptInfo(bot),
                'previous_action_outcomes': previous_action_outcomes,
                'current_goal_progress': current_goal_progress,
                'next_best_action': new_action
            }).content

            gate = findAndParseJsonLikeText(gate)[0]['gate']

            print(f'{gate=}')

            if 'false' in gate.lower():
                failed_action_proposal = next_best_action
                additional_info = gate['description']
                raise Exception("Gate check failed")

            # Get before
            previous_environment = env_info.getPromptInfo(bot)
            # Execute action
            # Note, change this to return a thing which we can add into the current environment type beat
            # ie change all control primitives to return a string of accomplishments
            try:
                exec(next_best_action)
            except:
                raise Exception('Action failed to execute')
            break
        except:
            continue

    changes_json = llms['summarize_environment_changes'].invoke({
        'goal': goal,
        'action': next_best_action,
        'previous_environment': previous_environment,
        'current_environment': env_info.getPromptInfo(bot),
        'previous_goal_progress': current_goal_progress
    }).content

    changes = findAndParseJsonLikeText(changes_json)[0]

    print(f'{changes=}')

    current_goal_progress = changes['new_goal_progress']
    environment_changes = changes['environment_changes']
    previous_action_outcomes.append([f'Action Number {len(previous_action_outcomes)}', new_action, 'Outcome:', environment_changes])

    # Try to gather any new memories
    new_memories = llms['gather_new_memories'].invoke({
        'current_environment': env_info.getPromptInfo(bot),
        'action': new_action,
        'environment_changes': environment_changes,
        'new_goal_progress': current_goal_progress
    }).content

    print(f'{new_memories=}')

    new_memories = findAndParseJsonLikeText(new_memories)[0]
    try:
        collection_name, memory = new_memories['memory1'].split(': ')[0], new_memories['memory1'].split(': ')[1]
        mem.store_memory(memory, collection_name)
        collection_name, memory = new_memories['memory2'].split(': ')[0], new_memories['memory2'].split(': ')[1]
        mem.store_memory(memory, collection_name)
    except:
        pass

perception_query='Are there any hostile mobs nearby that could pose a threat while collecting wood in the Forest biome at night?'
response='{\n    "scratchpad_for_thoughts": "Before proceeding, it would be beneficial to gather more information about the presence of hostile mobs in the Forest biome at night.",\n    "perception_function": "self.findBlockType(bot, \'mob\', 32)"\n}'
[['Attempt Number: 1', "self.findBlockType(bot, 'mob', 32)"]]
response='{\n    "scratchpad_for_thoughts": "Considering the potential risks of encountering hostile mobs while collecting wood at night in the Forest biome, it is crucial to gather more information to assess the situation accurately.",\n    "perception_function": "self.findBlockType(bot, \'mob\', 32)"\n}'
[['Attempt Number: 1', "self.findBlockType(bot, 'mob', 32)"], ['Attempt Number: 2', "self.findBlockType(bot, 'mob', 32)"]]


response='{\n    "scratchpad_for_thoughts": "Considering the previous attempts and the information available in the environment, it seems like there are no hostile mobs nearby based on the lack of mob sightings in the area. However, it\'s crucial to confirm this to ensure safety while collecting wood in the Forest biome at night.",\n    "COMPLETED": "Based on the information gathered from the environment, there are currently no hostile mobs nearby that could pose a threat while collecting wood in the Forest biome at night. The player\'s position is Vec3(-3.499839802991114, 72, 5.6030433548738445), and the environment_info shows no mobs present. It is safe to proceed with wood collection in the Forest biome at this time."\n}'
Completed task: ": "Based on the information gathered from the environment, there are currently no hostile mobs nearby that could pose a threat while collecting wood in the Forest biome at night. The player's position is Vec3(-3.499839802991114, 72, 5.6030433548738

response='{\n    "scratchpad_for_thoughts": "Based on the previous attempts, we have identified the location of oak logs and inquired about obstacles encountered on the path to collecting wood. To address the delay in deciding the path, we may need to consider factors such as time taken, efficiency of the chosen path, availability of tools, and the presence of hostile mobs.",\n    "COMPLETED": "The delay in deciding the path to collecting nearby wood could be attributed to factors such as lack of clear communication, conflicting priorities, uncertainty about the best approach, and external distractions. To address this delay, it is essential to track the time taken to reach the wood, count obstacles encountered, evaluate path efficiency, assess tool availability, consider hostile mobs, prioritize safety, and view the problem as part of a larger system in Minecraft."\n}'
Completed task: ": "The delay in deciding the path to collecting nearby wood could be attributed to factors such as l